# Spark SQL

This notebook will show some operation done previously using the DataFrame API but using SQL.

It also shows some functionality specific to SQL

## Creating a temporal view
A temporal view or a table needs to be registered to work with SQL statments. It can be created from an existing DataFrame

In [ ]:
df = spark.read.json('/work//data/flight-data/json/2015-summary.json')
df.show()

In [ ]:
df.createOrReplaceTempView('flights_view')


## First SQL statement
Once a temporal view is registered, it can be used to perform SQL statements. The output of a SQL transformation is always a DataFrame:

In [2]:
df = spark.sql('''
                SELECT DEST_COUNTRY_NAME, sum(count) as total
                FROM flights_view GROUP BY DEST_COUNTRY_NAME
            ''')
print(f'Result type: {type(df)}')
df.show(5)

Result type: <class 'pyspark.sql.dataframe.DataFrame'>


+-----------------+-----+
|DEST_COUNTRY_NAME|total|
+-----------------+-----+
|         Anguilla|   41|
|           Russia|  176|
|         Paraguay|   60|
|          Senegal|   40|
|           Sweden|  118|
+-----------------+-----+
only showing top 5 rows



DataFrame API's transformation can be mixed with SQL as follows:

In [3]:
spark.sql('''
            SELECT DEST_COUNTRY_NAME, sum(count)
            FROM flights_view GROUP BY DEST_COUNTRY_NAME
        ''').\
    where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10").\
    count()

12

## SQL statements

```
SELECT [ALL|DISTINCT] named_expression[, named_expression, ...]
    FROM relation[, relation, ...]
    [lateral_view[, lateral_view, ...]]
    [WHERE boolean_expression]
    [aggregation [HAVING boolean_expression]]
    [ORDER BY sort_expressions]
    [CLUSTER BY expressions]
    [DISTRIBUTE BY expressions]
    [SORT BY sort_expressions]
    [WINDOW named_window[, WINDOW named_window, ...]]
    [LIMIT num_rows]

named_expression:
    : expression [AS alias]

relation:
    | join_relation
    | (table_name|query|relation) [sample] [AS alias]
    : VALUES (expressions)[, (expressions), ...]
          [AS (column_name[, column_name, ...])]

expressions:
    : expression[, expression, ...]

sort_expressions:
    : expression [ASC|DESC][, expression [ASC|DESC], ...]
```

Select all and a new column:

In [4]:
spark.sql('''
    SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry
    FROM flights_view''').show(4)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
+-----------------+-------------------+-----+-------------+
only showing top 4 rows



Do some table-level aggregations:

In [5]:
spark.sql('''SELECT avg(count), count(distinct(DEST_COUNTRY_NAME)) FROM flights_view''').show()

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



Filtering:

In [6]:
spark.sql('SELECT * FROM flights_view WHERE count < 2 AND ORIGIN_COUNTRY_NAME != "Croatia"').show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
+-----------------+-------------------+-----+
only showing top 3 rows



Split

In [7]:
spark.sql('''
    SELECT SPLIT(DEST_COUNTRY_NAME, ' ') FROM flights_view''').show(5)

+-------------------------------+
|split(DEST_COUNTRY_NAME,  , -1)|
+-------------------------------+
|               [United, States]|
|               [United, States]|
|               [United, States]|
|                        [Egypt]|
|               [United, States]|
+-------------------------------+
only showing top 5 rows



Explode

In [8]:
spark.sql('''
    SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count, exploded
    FROM (SELECT *, SPLIT(DEST_COUNTRY_NAME, ' ') as splitted FROM flights_view)
    LATERAL VIEW explode(splitted) as exploded''').show(10)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|exploded|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|  United|
|    United States|            Romania|   15|  States|
|    United States|            Croatia|    1|  United|
|    United States|            Croatia|    1|  States|
|    United States|            Ireland|  344|  United|
|    United States|            Ireland|  344|  States|
|            Egypt|      United States|   15|   Egypt|
|    United States|              India|   62|  United|
|    United States|              India|   62|  States|
|    United States|          Singapore|    1|  United|
+-----------------+-------------------+-----+--------+
only showing top 10 rows



Retrieve a certain number of results only

In [9]:
spark.sql('SELECT DEST_COUNTRY_NAME FROM flights_view LIMIT 2').show()

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+



Getting distinct results:

In [10]:
spark.sql('SELECT DISTINCT ORIGIN_COUNTRY_NAME FROM flights_view').show(5)

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|           Paraguay|
|             Russia|
|           Anguilla|
|            Senegal|
|             Sweden|
+-------------------+
only showing top 5 rows



Sort

In [11]:
spark.sql('SELECT * FROM flights_view ORDER BY count DESC, DEST_COUNTRY_NAME ASC').show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



Grouping

In [12]:
spark.sql('''
    SELECT ORIGIN_COUNTRY_NAME, sum(count) AS total 
    FROM flights_view 
    GROUP BY ORIGIN_COUNTRY_NAME 
    ORDER BY total DESC''').show()

+-------------------+------+
|ORIGIN_COUNTRY_NAME| total|
+-------------------+------+
|      United States|411966|
|             Canada|  8483|
|             Mexico|  7187|
|     United Kingdom|  1970|
|              Japan|  1496|
| Dominican Republic|  1420|
|            Germany|  1336|
|        The Bahamas|   986|
|             France|   952|
|              China|   920|
|           Colombia|   867|
|        South Korea|   827|
|            Jamaica|   712|
|        Netherlands|   660|
|             Brazil|   619|
|         Costa Rica|   608|
|        El Salvador|   508|
|               Cuba|   478|
|             Panama|   465|
|              Spain|   442|
+-------------------+------+
only showing top 20 rows



Joins

In [13]:
# create a new table to join with
spark.createDataFrame([
    ('United States', "New York"),
    ('United States', "San Francisco"),
    ('Mexico', "Guadalajara"),
    ('Canda', 'Toronto')])\
  .toDF("COUNTRY", "CITY").createTempView('airports')

In [14]:
spark.sql('''
    SELECT * 
    FROM flights_view f 
    JOIN airports a
    ON f.ORIGIN_COUNTRY_NAME=a.COUNTRY
    ORDER BY a.COUNTRY''').show()

+--------------------+-------------------+-----+-------------+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|      COUNTRY|         CITY|
+--------------------+-------------------+-----+-------------+-------------+
|       United States|             Mexico| 7187|       Mexico|  Guadalajara|
|              Belize|      United States|  188|United States|San Francisco|
|Bonaire, Sint Eus...|      United States|   58|United States|San Francisco|
|             Uruguay|      United States|   43|United States|San Francisco|
|            Bulgaria|      United States|    3|United States|San Francisco|
|        Cook Islands|      United States|   13|United States|San Francisco|
|           Australia|      United States|  329|United States|San Francisco|
|               Chile|      United States|  174|United States|San Francisco|
|           Greenland|      United States|    2|United States|San Francisco|
|             Nigeria|      United States|   59|United States|San Francisco|

Case-When

In [15]:
spark.sql('''
    SELECT
        (CASE 
            WHEN DEST_COUNTRY_NAME = 'UNITED STATES' THEN 1
            WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
            ELSE -1 
        END) as condition
    FROM flights_view
''').show()

+---------+
|condition|
+---------+
|       -1|
|       -1|
|       -1|
|        0|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
|       -1|
+---------+
only showing top 20 rows



## Data Definition Functions

### Drop

Dropping a view (if exists)

In [16]:
spark.sql('DROP VIEW IF EXISTS flights_view')

23/05/13 17:55:02 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/05/13 17:55:02 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/05/13 17:55:06 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/05/13 17:55:06 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.20.0.7
23/05/13 17:55:07 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [17]:
# create the table again to re use it later
spark.read.json('/work//data/flight-data/json/2015-summary.json').\
    createOrReplaceTempView('flights_view')

The DataFrame is not dropped

In [18]:
df.show(5)

+-----------------+-----+
|DEST_COUNTRY_NAME|total|
+-----------------+-----+
|         Anguilla|   41|
|           Russia|  176|
|         Paraguay|   60|
|          Senegal|   40|
|           Sweden|  118|
+-----------------+-----+
only showing top 5 rows



Droping a table:

In [19]:
spark.sql('DROP TABLE IF EXISTS flights')

DataFrame[]

### Creating a table

A table lives across sessions

In [20]:
spark.sql('''
        CREATE TABLE flights (DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG) 
        USING JSON OPTIONS (path '/work/data/flight-data/json/2015-summary.json')
        ''')

23/05/13 17:55:10 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider JSON. Persisting data source table `default`.`flights` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
23/05/13 17:55:10 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/05/13 17:55:11 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/05/13 17:55:11 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/05/13 17:55:11 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [21]:
spark.sql('SELECT * FROM flights').show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



### Caching

In [22]:
spark.sql('CACHE TABLE flights_view')

DataFrame[]

And uncaching:

In [23]:
spark.sql('UNCACHE TABLE flights_view')

DataFrame[]

Or caching lazily:

In [24]:
spark.sql('CACHE LAZY TABLE flights')

DataFrame[]

In [25]:
spark.sql('SELECT * from flights').count()

256

### Show tables

In [26]:
spark.sql('SHOW TABLES').show()

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
| default|            flights|      false|
| default|           flights1|      false|
| default|       hive_flights|      false|
| default|just_usa_view_temp4|      false|
|        |           airports|       true|
|        |       flights_view|       true|
+--------+-------------------+-----------+



### Databases

In [27]:
spark.sql('CREATE DATABASE new_db')

23/05/13 17:55:13 WARN ObjectStore: Failed to get database new_db, returning NoSuchObjectException


DataFrame[]

Show the available databases

In [28]:
spark.sql('SHOW DATABASES').show()

+---------+
|namespace|
+---------+
|  default|
|   new_db|
+---------+



Which database is being used:

In [29]:
spark.sql('SELECT current_database()').show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



Switch to use another database:

In [30]:
spark.sql('USE new_db')

DataFrame[]

In [31]:
spark.sql('SELECT current_database()').show()

+------------------+
|current_database()|
+------------------+
|            new_db|
+------------------+



Drop a database

In [32]:
spark.sql('DROP DATABASE IF EXISTS new_db')

23/05/13 17:55:13 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist


DataFrame[]